In [49]:
import ipywidgets as w

In [68]:
intro = w.VBox(children=(w.HTML(value='<font size="+2"><b>CARA</b> Covid Calculator</font>'),
                         w.Text(placeholder="Simulation name"),
                         w.Text(placeholder="Room number")))

intro

In [74]:
volume_text = w.Text(placeholder="Room volume (m³)")
area_text = w.Text(placeholder="Room floor area (m²)")
height_text = w.Text(placeholder="Room ceiling height (m²)")

w.jslink((volume_text, 'value'), (area_text, 'disabled'))
w.jslink((volume_text, 'value'), (height_text, 'disabled'))
w.jslink((area_text, 'value'), (volume_text, 'disabled'))

w.VBox(children=(volume_text, w.Label(value="OR"),area_text, height_text), layout=w.Layout(align_items='center'))